In [1]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
train_it = datagen.flow_from_directory('ground_truth', class_mode='binary', batch_size=16, target_size=(320, 320))

Found 103426 images belonging to 2 classes.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(320, 320, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 501194380125769566
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5420531712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9802907836552221598
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [5]:
history = model.fit(train_it, epochs=10, steps_per_epoch=1_000)

Epoch 1/10
1000/1000 [==============================] - 58s 57ms/step - loss: 0.1629 - accuracy: 0.9442
Epoch 2/10
 277/1000 [=======>......................] - ETA: 42s - loss: 0.1156 - accuracy: 0.9596

KeyboardInterrupt: 

In [ ]:
# _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)

In [6]:
datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
train_it = datagen.flow_from_directory('ground_truth', class_mode='binary', batch_size=16, target_size=(224, 224))

Found 103426 images belonging to 2 classes.


In [11]:
from keras.applications import MobileNetV3Small

model2 = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=1,
    classifier_activation="softmax",
)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model2.fit(train_it, epochs=10)

Epoch 1/10
6465/6465 [==============================] - 545s 84ms/step - loss: 0.1496 - accuracy: 0.5384
Epoch 2/10
6465/6465 [==============================] - 555s 86ms/step - loss: 0.1120 - accuracy: 0.5384
Epoch 3/10
5521/6465 [========================>.....] - ETA: 1:21 - loss: 0.0886 - accuracy: 0.5385